In [4]:
from chat import chat

In [5]:
import sys
print(sys.executable)
import torch
print(torch.cuda.is_available())  # Should print True



/home/therkels/.conda/envs/myenv311/bin/python
True


In [3]:
import torch
from generate import generate
from transformers import AutoTokenizer, AutoModel

device = "cuda"
model = AutoModel.from_pretrained('GSAI-ML/LLaDA-1.5', trust_remote_code=True, dtype=torch.bfloat16).to(device).eval()
tokenizer = AutoTokenizer.from_pretrained('GSAI-ML/LLaDA-1.5', trust_remote_code=True)

gen_length = 128
steps = 128
conversation_num = 0
prompt = None

print('*' * 66)
print(f'**  Answer Length: {gen_length}  |  Sampling Steps: {steps}  **')

print('*' * 66)

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

******************************************************************
**  Answer Length: 128  |  Sampling Steps: 128  **
******************************************************************


In [7]:
user_input = "Generate SQL that is meaningful"

m = [{"role": "user", "content": user_input}]
user_input_tpl = tokenizer.apply_chat_template(m, add_generation_prompt=True, tokenize=False)
input_ids = tokenizer(user_input_tpl)['input_ids']
input_ids = torch.tensor(input_ids).to(device).unsqueeze(0)

if conversation_num == 0 or prompt is None:
    prompt = input_ids
else:
    prompt = torch.cat([prompt, input_ids[:, 1:],], dim=1)

out = generate(model, prompt, steps=steps, gen_length=gen_length, block_length=32, temperature=0., cfg_scale=0., remasking='low_confidence')

answer = tokenizer.batch_decode(out[:, prompt.shape[1]:], skip_special_tokens=True)[0]
print(f"Bot's reply: {answer}")

# Remove EOS
prompt = out[out != 126081].unsqueeze(0)
conversation_num += 1
print('-----------------------------------------------------------------------')

Bot's reply: Sure, I can help you generate meaningful SQL queries. Please provide the context or the specific problem you need the SQL to solve. For example, are you looking to retrieve data from a database, update a specific record, or perform some other operation?
-----------------------------------------------------------------------
